# Wire RAG

<a href="https://colab.research.google.com/github/appunite/Wire-RAG/blob/main/main_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

Install dependencies and input keys on Colab

In [ ]:
!pip install haystack-ai pinecone-haystack sentence-transformers pinecone transformers

In [6]:
import os
import getpass
os.environ["PINECONE_API_KEY"] = getpass.getpass("pinecone api key")
os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key")

Or load keys from .env file

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Populate Pinecone Database

### Scrape URLs

Whitelist: Allow any URL that begins with any element from the white_list.\
Blacklist: Block any URL that begins with any element from the black_list.

In [1]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import nest_asyncio

# Apply the nest_asyncio patch to allow nested event loops in Jupyter
nest_asyncio.apply()

# Asynchronous URL fetching with retry logic
async def fetch_urls(url_to_fetch: str, session: aiohttp.ClientSession, retries=3) -> set[str]:
    attempt = 0
    while attempt < retries:
        try:
            async with session.get(url_to_fetch, timeout=15) as response:
                if response.status != 200:
                    # print(f"Couldn't fetch {url} - status {response.status}")
                    return set()  # Return an empty set if the page doesn't load
                text_content = await response.text()
                soup = BeautifulSoup(text_content, "html.parser")
                urls = set(
                    urljoin(url_to_fetch, link['href'])
                    for link in soup.find_all('a', href=True)
                    if urlparse(urljoin(url_to_fetch, link['href'])).scheme in ('http', 'https')
                )
                # if attempt > 0: print(f"Successfully fetched URL: {url} on attempt {attempt + 1}")
                return urls
        except (aiohttp.ClientError, asyncio.TimeoutError):
            attempt += 1
            if attempt < retries:
                await asyncio.sleep(2 ** attempt)  # Exponential backoff
        except Exception as e:
            print(e)
            return set()

    print(f"Failed to fetch URL {u} after {retries} attempts.")
    return set()

# Check if URL should be allowed
def is_url_allowed(url_to_check: str, special_list: dict[str, list[str]] | None) -> bool:
    if special_list is None: return True
    
    # Return False if url is not in white_list
    if not any(url_to_check.startswith(i) for i in special_list["white_list"]): return False
    
    # Return False if url is in black_list 
    if any(url_to_check.startswith(i) for i in special_list["black_list"]): return False
    return True

# Asynchronous scraping with blacklist/whitelist and depth handling
async def scrape_urls(current_url: str, session: aiohttp.ClientSession, max_depth: int, current_depth=0, visited=None, list_data=None) -> list[str]:
    if visited is None:
        visited = []

    if current_depth > max_depth:
        return visited  # Stop recursion if depth limit exceeded

    visited.append(current_url)  # Store URL with its depth
    # Extract URLs from the current page
    urls = await fetch_urls(current_url, session)
    
    tasks = []
    for new_url in urls:
        if new_url not in visited and is_url_allowed(new_url, list_data):
            # Continue scraping at the next depth level if within depth limit
            if current_depth < max_depth:
                tasks.append(scrape_urls(new_url, session, max_depth, current_depth + 1, visited, list_data))

    # Await all the tasks concurrently
    await asyncio.gather(*tasks)
    return visited

# Entry point for asynchronous scraping
async def start_scraping(entry_url: str, depth: int, list_data=None) -> list[str]:
    async with aiohttp.ClientSession() as session:
        found_urls = await scrape_urls(entry_url, session, depth, list_data=list_data)
    return found_urls

In [3]:
starting_url = "https://docs.wire.com"
depth_limit = 2
filter_list = {"white_list": ["https://docs.wire.com"], "black_list": []}

scraped_urls = await start_scraping(starting_url, depth_limit, filter_list)

print(f"Total URLs found: {len(scraped_urls)}")

Total URLs found: 428


### Extract metadata and content

Prepare function to extract date based on regex patterns

In [4]:
import requests
import re
from datetime import datetime

# Regular expression patterns for various date formats
date_patterns = [
    r'\b\d{4}-\d{2}-\d{2}\b',            # Matches YYYY-MM-DD (e.g., 2024-09-20)
    r'\b\d{4}/\d{2}/\d{2}\b',            # Matches YYYY/MM/DD (e.g., 2024/09/20)
    r'\b\d{2}-\d{2}-\d{4}\b',            # Matches MM-DD-YYYY (e.g., 09-20-2024)
    r'\b\d{2}/\d{2}/\d{4}\b',            # Matches MM/DD/YYYY (e.g., 09/20/2024)
    r'\b\d{2}-\d{2}-\d{4}\b',            # Matches DD-MM-YYYY (e.g., 20-09-2024)
    r'\b\d{2}/\d{2}/\d{4}\b',            # Matches DD/MM/YYYY (e.g., 20/09/2024)
    r'\b\d{1,2}\s+[A-Za-z]+\s+\d{4}\b',  # Matches 20 September 2024
    r'\b[A-Za-z]+\s+\d{1,2},\s+\d{4}\b', # Matches September 20, 2024
    r'\b\d{1,2}\s+[A-Za-z]{3}\s+\d{4}\b', # Matches 20 Sep 2024 (short month)
    r'\b[A-Za-z]{3}\s+\d{1,2},\s+\d{4}\b', # Matches Sep 20, 2024 (short month)
    r'\b\d{1,2}-[A-Za-z]{3}-\d{4}\b',    # Matches 20-Sep-2024 (short month)
    r'\b[A-Za-z]{3}-\d{1,2}-\d{4}\b',    # Matches Sep-20-2024 (short month)
    r'\b[A-Za-z]+,\s+\d{1,2}\s+[A-Za-z]+\s+\d{4}\b', # Matches Friday, 20 September 2024 (with weekday)
    r'\b[A-Za-z]+,\s+[A-Za-z]+\s+\d{1,2},\s+\d{4}\b', # Matches Friday, September 20, 2024 (with weekday)
    r'\b\d{4}\.\d{2}\.\d{2}\b',          # Matches YYYY.MM.DD (e.g., 2024.09.20)
    r'\b\d{2}\.\d{2}\.\d{4}\b',          # Matches DD.MM.YYYY (e.g., 20.09.2024)
    r'\b\d{4}\s+[A-Za-z]{3}\s+\d{1,2}\b',# Matches 2024 Sep 20
    r'\b\d{4}\s+[A-Za-z]+\s+\d{1,2}\b',  # Matches 2024 September 20
    r'\b\d{1,2}\s+[A-Za-z]+,\s+\d{4}\b', # Matches 20 September, 2024 (comma after day)
    r'\b[A-Za-z]+\s+\d{1,2}\s+\d{4}\b'   # Matches September 20 2024 (no comma)
]


def find_all_dates_with_regex(text):
    """Search for all date matches using regex patterns."""
    dates = []
    for pattern in date_patterns:
        matches = re.findall(pattern, text)  # Find all matches for the pattern
        dates.extend(matches)
    return dates

def parse_date(date_str):
    """Convert a date string to a datetime object, trying different formats."""
    date_formats = [
        '%Y-%m-%d',        # Format: 2024-09-20
        '%Y/%m/%d',        # Format: 2024/09/20
        '%m-%d-%Y',        # Format: 09-20-2024
        '%m/%d/%Y',        # Format: 09/20/2024
        '%d-%m-%Y',        # Format: 20-09-2024
        '%d/%m/%Y',        # Format: 20/09/2024
        '%d %B %Y',        # Format: 20 September 2024
        '%B %d, %Y',       # Format: September 20, 2024
        '%d %b %Y',        # Format: 20 Sep 2024 (short month)
        '%b %d, %Y',       # Format: Sep 20, 2024 (short month)
        '%d-%b-%Y',        # Format: 20-Sep-2024 (short month)
        '%b-%d-%Y',        # Format: Sep-20-2024 (short month)
        '%A, %d %B %Y',    # Format: Friday, 20 September 2024 (with weekday)
        '%A, %B %d, %Y',   # Format: Friday, September 20, 2024 (with weekday)
        '%Y.%m.%d',        # Format: 2024.09.20
        '%d.%m.%Y',        # Format: 20.09.2024 (European format)
        '%Y %b %d',        # Format: 2024 Sep 20 (short month)
        '%Y %B %d',        # Format: 2024 September 20
        '%d %B, %Y',       # Format: 20 September, 2024 (comma after day)
        '%B %d %Y'         # Format: September 20 2024 (no comma)
    ]
    
    for date_format in date_formats:
        try:
            return datetime.strptime(date_str, date_format)
        except ValueError:
            pass
    return None

def get_latest_date(dates):
    """Return the latest valid date from a list of date strings."""
    parsed_dates = [parse_date(date) for date in dates]
    # Filter out any dates that couldn't be parsed and return the latest one
    valid_dates = [d for d in parsed_dates if d is not None]
    return max(valid_dates).strftime('%Y-%m-%d') if valid_dates else "None"

def extract_date(my_soup: BeautifulSoup) -> str:
    # Extract all text content to search for a date pattern
    text_content = my_soup.get_text(separator=' ', strip=True)

    # Use regex to find all dates in the page's text content
    dates = find_all_dates_with_regex(text_content)

    # Get the latest date found
    latest_date = get_latest_date(dates)
    return latest_date

Prepare function to extract content and metadata from a URL

In [5]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


def extract_content_and_metadata(url: str) -> list[dict] | None:
    try:
        # Use retry session for robust requests
        session = requests_retry_session()
        response = session.get(url, timeout=10)

        # Check if request was successful
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.title.string if soup.title else "None"
        date = extract_date(soup)
        
        content_by_headline = {}
        current_header = None

        # Loop through the elements, keeping track of headlines and paragraphs
        for element in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'ul', 'ol']):
            if element.name.startswith('h'):
                # Headline
                current_header = element.get_text(strip=True)
                current_header = current_header.replace("\n", "")
                current_header = re.sub(r"\s+", " ", current_header)

            elif element.name in ['p', 'ul', 'ol'] and current_header:
                # Append the text under the last seen headline
                value = content_by_headline[current_header] if current_header in content_by_headline else ""
                content_by_headline[current_header] = f"{value} {element.get_text(strip=True).replace('\n', ' ')}"
    
        result_dicts = []
        for headline, content in content_by_headline.items():
            result_dicts.append({"content": content, "metadata": {"url": url, "title": title, "headline": headline, "date": date}})
            
        return result_dicts

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None


In [6]:
all_documents = []
for u in scraped_urls:
    all_documents += extract_content_and_metadata(u)
print(len(all_documents))

6670


To delete all records u need to `pip install "pinecone[grpc]"` and run the following code.

In [9]:
from dotenv import load_dotenv
from pinecone.grpc import PineconeGRPC as Pinecone
import os

load_dotenv()

# Uncomment to delete all db records
# Pinecone(api_key=os.getenv("PINECONE_API_KEY")).Index("default").delete(delete_all=True, namespace='default')

True

Initialize Pinecone Document Store

In [7]:
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack import Pipeline
from haystack import Document
from haystack_integrations.document_stores.pinecone import PineconeDocumentStore

document_store = PineconeDocumentStore(
    index="default",
    namespace="default",
    dimension=384,
    metric="cosine",
    spec={"serverless": {"region": "us-east-1", "cloud": "aws"}}
)

ready_list = []
for doc in all_documents:
    ready_list.append(Document(content=doc["content"], meta=doc["metadata"]))

Prepare pipeline components

In [39]:
document_cleaner = DocumentCleaner()
# For all-MiniLM-L6-v2 input text longer than 256 word pieces is truncated.
document_splitter = DocumentSplitter(split_by="word", split_length=256, split_overlap=20)
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store)

Create a pipeline to populate the Pinecone Document Store with test case documents

In [40]:
preprocessing_pipeline = Pipeline()

preprocessing_pipeline.add_component(instance=document_cleaner, name="document_cleaner")
preprocessing_pipeline.add_component(instance=document_splitter, name="document_splitter")
preprocessing_pipeline.add_component(instance=document_embedder, name="document_embedder")
preprocessing_pipeline.add_component(instance=document_writer, name="document_writer")

preprocessing_pipeline.connect("document_cleaner", "document_splitter")
preprocessing_pipeline.connect("document_splitter", "document_embedder")
preprocessing_pipeline.connect("document_embedder", "document_writer")

🚅 Components
  - document_cleaner: DocumentCleaner
  - document_splitter: DocumentSplitter
  - document_embedder: SentenceTransformersDocumentEmbedder
  - document_writer: DocumentWriter
🛤️ Connections
  - document_cleaner.documents -> document_splitter.documents (List[Document])
  - document_splitter.documents -> document_embedder.documents (List[Document])
  - document_embedder.documents -> document_writer.documents (List[Document])

Run the pipeline

In [ ]:
preprocessing_pipeline.run(data = {"document_joiner": { "documents" : ready_list }})
# preprocessing_pipeline.inputs()
preprocessing_pipeline.show()

## Test RAG with Pinecone Document Store

Restart the kernel and run the following code to test the RAG pipeline with the populated Pinecone Document Store.

Create pipeline to run a query

In [5]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.components.retrievers.pinecone import PineconeEmbeddingRetriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack import Pipeline
from haystack_integrations.document_stores.pinecone import PineconeDocumentStore
from dotenv import load_dotenv

template = """You are a knowledgeable assistant responsible for creating comprehensive documentation based on the following list of documents that refer to the user’s question. The content of these documents may contradict each other, so please prioritize the information from the documents with the most recent dates. If there are contradicting documents with dates labeled as 'None', provide all alternatives and explicitly indicate which parts contradict one another. However, if a document with date labeled as 'None' does not conflict with others, it should be included without special mention.

Instructions:
1. Analyze the Documents:
   - Review each document, noting any conflicting information.
   - Prioritize information from the most recent documents.
2. Handling Documents with 'None' Date:
   - If a document has a date marked as 'None':
     - Include all relevant alternatives and clearly indicate contradictions.
     - If it does not conflict with other documents, include it without special mention.

Output Format:
Your output should be structured using Markdown and include the following sections:
1. Summary:
   - Provide a brief overview of the key findings from all documents.
2. Detailed Analysis:
   - Present detailed descriptions of key points, prioritizing the latest information.
   - Preserve and format any code snippets from the documents appropriately.
3. Contradictions:
   - For documents dated 'None', list all relevant alternatives and explicitly highlight any contradictions.

General Guidelines:
- Ensure thoroughness by including all relevant information, aiming for completeness rather than brevity.
- Use headings, lists, and code blocks to enhance readability and organization.

User Question: {{question}}
Documents to Analyze:
{% for doc in documents %}
Date: {{doc.meta['date']}}
Title: {{doc.meta['title']}} - {{doc.meta['headline']}}
Content: {{doc.content}}
{% endfor %}"""

load_dotenv()

document_store = PineconeDocumentStore(
    index="default",
    namespace="default",
    dimension=384,
    metric="cosine",
    spec={"serverless": {"region": "us-east-1", "cloud": "aws"}}
)

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = PineconeEmbeddingRetriever(document_store=document_store, top_k=20)
prompt_builder = PromptBuilder(template=template)
generator = OpenAIGenerator(model="gpt-4o")
answer_builder = AnswerBuilder()

rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", text_embedder)
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("generator", generator)
rag_pipeline.add_component("answer_builder", answer_builder)

rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("retriever", "answer_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")
rag_pipeline.connect("generator.replies", "answer_builder.replies")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: PineconeEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: OpenAIGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> answer_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)
  - generator.replies -> answer_builder.replies (List[str])

Run the pipeline with a query

In [6]:
query = "Generate full documentation of stern/ Backoffice tool (basic Swagger based interface)"
result = rag_pipeline.run({
    "text_embedder": {"text": query},
    "prompt_builder": {"question": query},
    "answer_builder": {"query": query}
})

print(result['answer_builder']['answers'][0].query)
print(result['answer_builder']['answers'][0].data)
print(result['answer_builder']['answers'][0].documents)

with open("./Data/Outputs/output.md", "w") as f:
    f.write(result['answer_builder']['answers'][0].data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generate full documentation of stern/ Backoffice tool (basic Swagger based interface)
# Stern/Backoffice Tool Documentation

## Summary
This documentation provides an overview of the processes and tools used within the stern/Backoffice tool for generating Swagger-based API documentation. The latest methods, libraries, and integration points are highlighted, with a special focus on the implementation of `servant-swagger-ui` for local Swagger documentation, feature flags, and various SCIM-related documentations.

## Detailed Analysis

### Latest Information from 2024-07-09 Documents

- **Integration of `servant-swagger-ui`**:
    - The `servant-swagger-ui` has been integrated with the `brig` service to facilitate easier interaction with API documentation.
    - This enables local viewing of Swagger documentation, ensuring users can explore, make requests, and generate client code from the API documentation interactively.

- **SCIM Documentation**:
    - Features related to SCIM, such as 